<a href="https://colab.research.google.com/github/navroz-lamba/DS-Unit-2-Kaggle-Challenge/blob/master/assignment%20-%20224.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# to be able to view all the columns in the head 
pd.set_option('display.max_columns', None)

train_df = pd.merge(pd.read_csv('https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/waterpumps/train_features.csv'), 
                 pd.read_csv('https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/waterpumps/train_labels.csv')).set_index('id')

X_test = pd.read_csv('https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/waterpumps/test_features.csv', index_col='id')

sample_submission = pd.read_csv('https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/waterpumps/sample_submission.csv')

train_df.shape, X_test.shape

((59400, 40), (14358, 39))

In [ ]:
hc_cols_99 = ['funder', 'installer', 'subvillage', 'lga']
    # to iterate over every column in hc_cols_99
for value in hc_cols_99:
    # list of top 99 values
    top99 = train_df[value].value_counts()[:99].index
    # if not in top 99 put it under 'other'
    train_df.loc[~train_df[value].isin(top99), value] = 'Other'

In [ ]:
# also, dropping 'num_private' as 90% or more are 0's and recorded_by as its unique and won't add any value

# lets make a list of cols we will be dropping so we could call it in the function 
train_df = train_df.drop(['waterpoint_type_group', 'source', 'source_class','quantity_group', 'extraction_type_group', 
                        'extraction_type_class', 'region_code', 'quality_group', 'payment_type', 'recorded_by', 'num_private'], axis =1)

X_test= X_test.drop(['waterpoint_type_group', 'source', 'source_class','quantity_group', 'extraction_type_group', 
                        'extraction_type_class', 'region_code', 'quality_group', 'payment_type', 'recorded_by', 'num_private'], axis =1)

In [ ]:
import numpy as np

def wrangle(X):
    
    
    # make a copy 
    X = X.copy()

    
    # replace the funky measurements with 0
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    
    # Replace all cols with 0s with Nan
    cols_with_zeros = ['longitude', 'latitude', 'construction_year']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
    
    # replace Nan using mean for lat and lon 
    X['longitude'] = X['longitude'].fillna(X['longitude'].mean())
    X['latitude'] = X['latitude'].fillna(X['latitude'].mean())
    
    
    # fillna for construction year with bfill 
    X['construction_year'] = X['construction_year'].fillna(X['construction_year'].median())
    # change the dtype of date_recorded so we could extract year 
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format = True)
    # Extracting year and assigning a new column
    X['year_recorded'] = X['date_recorded'].dt.year
    # making a new column, time since inspection
    X['time_since_inspection'] = X['year_recorded'] - X['construction_year']

    
    # convert district code and construction year to object 
    X['district_code'] = X['district_code'].astype('object')
    X['year_recorded'] = X['year_recorded'].astype('object')
    
    X['time_since_inspection'] = X['time_since_inspection'].astype(int)

    
#     X['num_private'] = np.sqrt(X['num_private'])
    # Drop the following
#     X = X.drop(['date_recorded', 'year_recorded'], axis=1)
#     X = X.drop(['date_recorded', 'year_recorded', 'contruction_year'], drop_cols, axis=1)

    # taking median of the pop instead of zeros
    X['population'] = X['population'].replace(0, np.nan)
    X['population'] = X['population'].fillna(X['population'].median())
    
    X['amount_tsh'] = X['amount_tsh'].replace(0, np.nan)
    X['amount_tsh'] = X['amount_tsh'].fillna(X['amount_tsh'].median())
    
#     X['gps_height'] = X['gps_height'].replace(0, np.nan)
#     X['gps_height'] = X['gps_height'].fillna(X['amount_tsh'].median())
    
        
    
    return X

# apply on train and test 
train_df = wrangle(train_df)
X_test = wrangle(X_test)


# missing values

In [ ]:
# replace '0' in funder and installer with 'None'
train_df['funder'] = train_df['funder'].replace('0', 'None')
train_df['installer'] = train_df['installer'].replace('0', 'None')
train_df["scheme_management"].fillna("unknown", inplace = True)
train_df["public_meeting"].fillna("unknown", inplace = True)
train_df["permit"].fillna("unknown", inplace = True)
train_df["funder"].fillna("unknown", inplace = True)
train_df["installer"].fillna("unknown", inplace = True)

X_test['funder'] = X_test['funder'].replace('0', 'None')
X_test['installer'] = X_test['installer'].replace('0', 'None')
X_test["scheme_management"].fillna("unknown", inplace = True)
X_test["public_meeting"].fillna("unknown", inplace = True)
X_test["permit"].fillna("unknown", inplace = True)
X_test["funder"].fillna("unknown", inplace = True)
X_test["installer"].fillna("unknown", inplace = True)

In [ ]:
# ordinal for Public meeting 
order_dict_pub_meet = {True:2,False:1,"unknown":0}
train_df["public_meeting_code"] = [order_dict_pub_meet[item] for item in train_df.public_meeting] 
del train_df["public_meeting"]

X_test["public_meeting_code"] = [order_dict_pub_meet[item] for item in X_test.public_meeting] 
del X_test["public_meeting"]

# ordinal for Permit
order_dict_permit = {True:2,False:1,"unknown":0}
train_df["permit_code"] = [order_dict_pub_meet[item] for item in train_df.permit] 
del train_df["permit"]

X_test["permit_code"] = [order_dict_pub_meet[item] for item in X_test.permit] 
del X_test["permit"]

In [ ]:
def cardinalilty_fix(X):
    
# fetching the high cordanility columns which we will reduce to 99 unique values
    hc_cols_99 = ['funder', 'installer', 'subvillage', 'lga']
    # to iterate over every column in hc_cols_99
    for value in hc_cols_99:
    # list of top 99 values
        top99 = X[value].value_counts()[:99].index
    # if not in top 99 put it under 'other'
        X.loc[~X[value].isin(top99), value] = 'Other'
    
    # fetching the high cordanility columns which we will reduce to 30 unique values
    hc_cols_30 = ['ward', 'scheme_name']
    # to iterate over every column in hc_cols_99
    for value in hc_cols_30:
    # list of top 99 values
        top30 = X[value].value_counts()[:30].index
    # if not in top 99 put it under 'other'
        X.loc[~X[value].isin(top30), value] = 'Other'
    
    # fetching the high cordanility columns which we will reduce to 30 unique values
    hc_cols_20 = ['wpt_name']
    # to iterate over every column in hc_cols_99
    for value in hc_cols_20:
    # list of top 99 values
        top20 = X[value].value_counts()[:20].index
    # if not in top 99 put it under 'other'
        X.loc[~X[value].isin(top20), value] = 'Other'
        
        
    # we will use frequency encoding for these nominal cat columns
    freq_cat_col = ['funder', 'installer', 'subvillage', 'lga']
    for col in freq_cat_col:
        frequency_map = (X[col].value_counts() / len(X)).to_dict()
        X[col] = X[col].map(frequency_map)
    
    
    return X

# apply on train and test 
train_df = cardinalilty_fix(train_df)
X_test = cardinalilty_fix(X_test)

In [ ]:
X_test.isna().sum()

amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
basin                    0
subvillage               0
region                   0
district_code            0
lga                      0
ward                     0
population               0
scheme_management        0
scheme_name              0
construction_year        0
extraction_type          0
management               0
management_group         0
payment                  0
water_quality            0
quantity                 0
source_type              0
waterpoint_type          0
year_recorded            0
time_since_inspection    0
public_meeting_code      0
permit_code              0
dtype: int64

In [ ]:
X_test.shape

(14358, 30)

# feature engineering 


In [ ]:
#take the recorded date of the water point’s last known observation and 
# subtract the year of the construction 

train_df['age_of_the_waterpoint'] = ((train_df['date_recorded'].dt.year).max() - train_df['construction_year']).astype(int)

X_test['age_of_the_waterpoint'] = ((X_test['date_recorded'].dt.year).max() - X_test['construction_year']).astype(int)


In [ ]:
# make new features with population and age of the water pump
train_df['pop_age_m'] = np.sqrt((train_df['population'] * train_df['age_of_the_waterpoint'])).astype('object')
train_df['pop_age_ratio'] = (train_df['population'] / train_df['age_of_the_waterpoint']).astype('object')
X_test['pop_age_m'] = np.sqrt((X_test['population'] * X_test['age_of_the_waterpoint'])).astype('object')
X_test['pop_age_ratio'] = (X_test['population'] / X_test['age_of_the_waterpoint']).astype('object')

In [ ]:
train_df.isna().sum()

amount_tsh               0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
basin                    0
subvillage               0
region                   0
district_code            0
lga                      0
ward                     0
population               0
scheme_management        0
scheme_name              0
construction_year        0
extraction_type          0
management               0
management_group         0
payment                  0
water_quality            0
quantity                 0
source_type              0
waterpoint_type          0
status_group             0
time_since_inspection    0
public_meeting_code      0
permit_code              0
age_of_the_waterpoint    0
days_since_recorded      0
pop_age_m                0
pop_age_ratio            0
pop_time_m               0
dtype: int64

In [ ]:

# make new features with population and age of the water pump
train_df['pop_time_m'] = train_df['population'] * train_df['time_since_inspection'].astype('object')
# train_df['pop_time_ratio'] = train_df['population'] / train_df['time_since_inspection'].astype('object')

X_test['pop_time_m'] = X_test['population'] * X_test['time_since_inspection'].astype('object')
# X_test['pop_time_ratio'] = X_test['population'] / X_test['time_since_inspection'].astype('object')

In [ ]:
# removing outliers
train_df = train_df[(train_df['amount_tsh'] >= np.percentile(train_df['amount_tsh'], 0.5)) & 
        (train_df['amount_tsh'] <= np.percentile(train_df['amount_tsh'], 99.5)) & 
        (train_df['latitude'] >= np.percentile(train_df['latitude'], 0.05)) & 
        (train_df['latitude'] < np.percentile(train_df['latitude'], 99.95)) &
        (train_df['longitude'] >= np.percentile(train_df['longitude'], 0.05)) & 
        (train_df['longitude'] <= np.percentile(train_df['longitude'], 99.95))]

# X_test = X_test[(X_test['amount_tsh'] >= np.percentile(X_test['amount_tsh'], 0.5)) & 
#         (X_test['amount_tsh'] <= np.percentile(X_test['amount_tsh'], 99.5)) & 
#         (X_test['latitude'] >= np.percentile(X_test['latitude'], 0.05)) & 
#         (X_test['latitude'] < np.percentile(X_test['latitude'], 99.95)) &
#         (X_test['longitude'] >= np.percentile(X_test['longitude'], 0.05)) & 
#         (X_test['longitude'] <= np.percentile(X_test['longitude'], 99.95))]

In [ ]:
def construction_wrangler(X):
    if X['construction_year'] >= 1960 and X['construction_year'] < 1970:
        return '60s'
    elif X['construction_year'] >= 1970 and X['construction_year'] < 1980:
        return '70s'
    elif X['construction_year'] >= 1980 and X['construction_year'] < 1990:
        return '80s'
    elif X['construction_year'] >= 1990 and X['construction_year'] < 2000:
        return '90s'
    elif X['construction_year'] >= 2000 and X['construction_year'] < 2010:
        return '00s'
    elif X['construction_year'] >= 2010:
        return '10s'
    else:
        return 'unknown'
    
train_df['construction_year'] = train_df.apply(lambda X: construction_wrangler(X), axis=1)
X_test['construction_year'] = X_test.apply(lambda X: construction_wrangler(X), axis=1)

In [ ]:
# calculate the number of days past since the date_recorded of a particular pump till the most recent date of the dataset


train_df['days_since_recorded'] = pd.datetime(2013, 12, 3) - pd.to_datetime(train_df.date_recorded)
train_df['days_since_recorded'] = train_df['days_since_recorded'].astype('timedelta64[D]').astype(int)

X_test['days_since_recorded'] = pd.datetime(2013, 12, 3) - pd.to_datetime(X_test.date_recorded)
X_test['days_since_recorded'] = X_test['days_since_recorded'].astype('timedelta64[D]').astype(int)

/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  after removing the cwd from sys.path.
/Users/navrozlamba/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  import sys


In [ ]:
# drop date columsn from both train and test 
def drop_date(X):
    
    X = X.drop(['pop_time_m'], axis=1)
#      X = X.drop(['date_recorded', 'year_recorded', 'pop_time_m'], axis=1)
    return X
# apply to both 
train_df = drop_date(train_df)
X_test = drop_date(X_test)

In [ ]:
train_df.isna().sum()

amount_tsh               0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
basin                    0
subvillage               0
region                   0
district_code            0
lga                      0
ward                     0
population               0
scheme_management        0
scheme_name              0
construction_year        0
extraction_type          0
management               0
management_group         0
payment                  0
water_quality            0
quantity                 0
source_type              0
waterpoint_type          0
status_group             0
time_since_inspection    0
public_meeting_code      0
permit_code              0
age_of_the_waterpoint    0
days_since_recorded      0
dtype: int64

# SPLIT 

In [ ]:
y_train = train_df['status_group']
X_train= train_df.drop(['status_group'], axis=1)

In [ ]:
# cat attributes 
cat_attribs = X_train.select_dtypes(include='object')
cat_attribs.columns

Index(['wpt_name', 'basin', 'region', 'district_code', 'ward',
       'scheme_management', 'scheme_name', 'construction_year',
       'extraction_type', 'management', 'management_group', 'payment',
       'water_quality', 'quantity', 'source_type', 'waterpoint_type',
       'pop_age_m', 'pop_age_ratio'],
      dtype='object')

In [ ]:
# # Train-validation split
# from sklearn.model_selection import train_test_split

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=11)

# first run - original notebook 
-max_depth=20, n_estimators=12, k=28 
- Training MAE: 0.9679447820287523
- Training MAE: 0.8091971544715447

# second run - 
-max_depth=, n_estimators=, k= 
- Training MAE: 
- Training MAE: 

In [ ]:
# from sklearn.pipeline import make_pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler
# # from sklearn.preprocessing import OrdinalEncoder
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from category_encoders import OrdinalEncoder
# from sklearn.feature_selection import SelectKBest

# tree_model = make_pipeline(
#     OrdinalEncoder(cols = cat_attribs),
#     SelectKBest(k=28),
#     RandomForestClassifier(random_state=11, max_depth=20, n_estimators=121) # min_impurity_decrease=0.000081
# )

# tree_model.fit(X_train, y_train)

In [ ]:
# from sklearn.metrics import accuracy_score
# print('Training MAE:', accuracy_score(y_train, tree_model.predict(X_train)))
# print('Training MAE:', accuracy_score(y_val, tree_model.predict(X_val)))

# Regularizing 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from category_encoders import OrdinalEncoder
from sklearn.feature_selection import SelectKBest

tree_model = make_pipeline(
    OrdinalEncoder(cols = cat_attribs),
    SelectKBest(),
    RandomForestClassifier(random_state=11))

params = {'selectkbest__k': range(30,34),
          'randomforestclassifier__n_estimators': range(114,119),
         'randomforestclassifier__max_depth': range(20,23)}

rand = RandomizedSearchCV(tree_model,
                  params,
                  verbose=1,
                  cv=7,
                  n_jobs=-1)

In [ ]:
rand.fit(X_train, y_train)

Fitting 7 folds for each of 10 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:  8.1min finished


RandomizedSearchCV(cv=7,
                   estimator=Pipeline(steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=       wpt_name                    basin       region district_code   ward  \
id                                                                           
69572      none               Lake Nyasa       Iringa             5  Other   
8776   Zahanati            Lake Victoria         Mara             2  Other   
34310     Other                  Pangani      Manyara             4  Other   
67743     Other  Ruvuma / Southern Coast       Mtwara            63  Other   
19728   Shuleni            Lake Victoria       Kagera             1  Other   
...         ...                      ...          ...           ...    ...   
60739     Other                  Pangani  Kilimanjaro             5  Oth...
37057   44.1588       11.5385       1650  
31282   44.1588       11.5385       1650  
26348   40.6202       13.6364       1350  

[59019 rows x 1

In [ ]:
print(rand.best_score_)
print(rand.best_params_)
print(rand.best_estimator_)

0.811179446124278
{'selectkbest__k': 33, 'randomforestclassifier__n_estimators': 116, 'randomforestclassifier__max_depth': 22}
Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=       wpt_name                    basin       region district_code   ward  \
id                                                                           
69572      none               Lake Nyasa       Iringa             5  Other   
8776   Zahanati            Lake Victoria         Mara             2  Other   
34310     Other                  Pangani      Manyara             4  Other   
67743     Other  Ruvuma / Southern Coast       Mtwara            63  Other   
19728   Shuleni            Lake Victoria       Kagera             1  Other   
...         ...                      ...          ...           ...    ...   
60739     Other                  Pangani  Kilimanjaro             5  Other   
27263     Other                   Rufiji       Iringa             4  Other   
3705...
197.000000   

# one final step 

In [ ]:
model = rand.best_estimator_

model.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=       wpt_name                    basin       region district_code   ward  \
id                                                                           
69572      none               Lake Nyasa       Iringa             5  Other   
8776   Zahanati            Lake Victoria         Mara             2  Other   
34310     Other                  Pangani      Manyara             4  Other   
67743     Other  Ruvuma / Southern Coast       Mtwara            63  Other   
19728   Shuleni            Lake Victoria       Kagera             1  Other   
...         ...                      ...          ...           ...    ...   
60739     Other                  Pangani  Kilimanjaro             5  Other   
27263     Other                   Rufiji       Iringa             4  Other   
3705...
borehole                4
other                   5
shallow well            6
river/lake              7
NaN                    -2
dtype: int

In [ ]:
y_pred = model.predict(X_test) 

In [ ]:
y_pred = pd.DataFrame(y_pred, columns=['status_group'], index=X_test.index)

In [ ]:
y_pred.to_csv('2020-08-13_2Submission.csv')

In [ ]:
# y_pred.shape

# try 2

In [ ]:
# tree_model = make_pipeline(
#     OrdinalEncoder(cols = cat_attribs),
#     SelectKBest(k=28),
#     RandomForestClassifier(random_state=11, max_depth=23, n_estimators=123) # min_impurity_decrease=0.000081
# )
# tree_model.fit(X_train, y_train)

In [ ]:
# y_pred = model.predict(X_test) 

# y_pred = pd.DataFrame(y_pred, columns=['status_group'], index=X_test.index)

# y_pred.to_csv('2020-08-13_Submission.csv')